In [17]:
import os
import pandas as pd

# Directory containing the CSV files
directory = "./Wine_Stats/"

# Initialize variables
files = os.listdir(directory)
dataframes = []
rows = 0

# Process each file
for file in files:
    if file.endswith(".csv"):  # Ensure only CSV files are processed
        file_path = os.path.join(directory, file)
        try:
            df = pd.read_csv(file_path)
            rows += len(df)
            dataframes.append(df)
        except Exception as e:
            print(f"Error processing file {file}: {e}")

# Concatenate all DataFrames into one
if dataframes:  # Check if any DataFrames were added
    main = pd.concat(dataframes, ignore_index=True)
    print(f"Number of rows processed: {rows}")
    print(f"Length of the main DataFrame: {len(main)}")
    main.to_csv("wine_df.csv", index=False)
else:
    print("Error: No valid CSV files found in the directory. Please check the files in './Wine_Stats/'.")


Number of rows processed: 5145
Length of the main DataFrame: 5145


In [19]:
# Initial data exploration and cleaning
if 'main' in locals() or 'main' in globals():
    print("DataFrame Info:")
    print(main.info())  # Shows data types, non-null counts, and memory usage

    # Remove rows with null values
    main.dropna(inplace=True)

    # Remove duplicate rows
    main.drop_duplicates(inplace=True)

    # Save the cleaned DataFrame to a CSV file
    main.to_csv("project_cleaned.csv", index=False)
    print("Cleaned DataFrame saved to 'project_cleaned.csv'")
else:
    print("Error: 'main' DataFrame is not defined. Please ensure the DataFrame is created before running this code.")

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5145 entries, 0 to 5144
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5145 non-null   int64  
 1   Name               5145 non-null   object 
 2   Rating             5145 non-null   float64
 3   Number of Ratings  5145 non-null   int64  
 4   Price              5145 non-null   float64
 5   Region             5145 non-null   object 
 6   Winery             5142 non-null   object 
 7   Wine style         4624 non-null   object 
 8   Alcohol content    5145 non-null   float64
 9   Grapes             2977 non-null   object 
 10  Food pairings      5145 non-null   object 
 11  Bold               5145 non-null   float64
 12  Tannin             5145 non-null   float64
 13  Sweet              5145 non-null   float64
 14  Acidic             5145 non-null   float64
dtypes: float64(7), int64(2), object(6)
memory usage: 603.1+ 

In [13]:
# Handle outliers and missing values
if 'main' in locals() or 'main' in globals():
    # Step 1: Detect Missing Values
    print("Missing Values Count Before Imputation:")
    print(main.isnull().sum())

    # Fill missing numerical values with the mean
    for col in main.select_dtypes(include=["float", "int"]).columns:
        if main[col].isnull().sum() > 0:  # Check for missing values
            print(f"Filling missing values in '{col}' with mean ({main[col].mean()})")
            main[col].fillna(main[col].mean(), inplace=True)

    # Step 2: Detect and Handle Outliers
    def remove_outliers(df, col):
        Q1 = df[col].quantile(0.25)  # First quartile
        Q3 = df[col].quantile(0.75)  # Third quartile
        IQR = Q3 - Q1                # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        initial_length = len(df)
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        print(f"Removed outliers from '{col}'. Rows reduced from {initial_length} to {len(df)}.")
        return df

    # Apply outlier removal to numeric columns
    for col in main.select_dtypes(include=["float", "int"]).columns:
        main = remove_outliers(main, col)

    # Final check
    print("Missing Values Count After Imputation:")
    print(main.isnull().sum())
    print("DataFrame shape after handling outliers and missing values:", main.shape)
else:
    print("Error: 'main' DataFrame is not defined.")

Missing Values Count Before Imputation:
Unnamed: 0           0
Name                 0
Rating               0
Number of Ratings    0
Price                0
Region               0
Winery               0
Wine style           0
Alcohol content      0
Grapes               0
Food pairings        0
Bold                 0
Tannin               0
Sweet                0
Acidic               0
dtype: int64
Removed outliers from 'Unnamed: 0'. Rows reduced from 2977 to 2977.
Removed outliers from 'Rating'. Rows reduced from 2977 to 2941.
Removed outliers from 'Number of Ratings'. Rows reduced from 2941 to 2627.
Removed outliers from 'Price'. Rows reduced from 2627 to 2587.
Removed outliers from 'Alcohol content'. Rows reduced from 2587 to 2468.
Removed outliers from 'Bold'. Rows reduced from 2468 to 2468.
Removed outliers from 'Tannin'. Rows reduced from 2468 to 1935.
Removed outliers from 'Sweet'. Rows reduced from 1935 to 1835.
Removed outliers from 'Acidic'. Rows reduced from 1835 to 1835.
Missin

In [21]:
# Adding new columns to the DataFrame
if 'main' in locals() or 'main' in globals():
    # Check if 'region' exists in the DataFrame
    if 'region' in main.columns:
        # Adding new columns for Country and Country_region
        main[['Country', 'Country_region']] = main['region'].str.split(' - ', expand=True)
    else:
        print("Error: 'region' column not found in the DataFrame.")

    # Ensure 'Food pairings' column exists
    if 'Food pairings' in main.columns:
        # Assuming 'Food pairings' contains values in list format in string representation
        food_pairings = main['Food pairings'].apply(eval)  # Convert strings to lists

        # Flatten the list of food pairings
        all_foods = set(food for sublist in food_pairings for food in sublist)

        # Create dummy columns for each unique food pairing
        for food in all_foods:
            main[food] = main['Food pairings'].apply(lambda x: food in eval(x))
    else:
        print("Error: 'Food pairings' column not found in the DataFrame.")

    # Save the updated DataFrame
    main.to_csv("project_final_with_columns.csv", index=False)
    print("Final DataFrame with new columns saved to 'project_final_with_columns.csv'")
else:
    print("Error: 'main' DataFrame is not defined.")

Error: 'region' column not found in the DataFrame.
Final DataFrame with new columns saved to 'project_final_with_columns.csv'


In [23]:
# Column removal
if 'main' in locals() or 'main' in globals():
    columns_to_drop = []

    # 1. Check for high missing value columns (e.g., > 50% missing values)
    high_missing_columns = main.columns[main.isnull().mean() > 0.5]
    if not high_missing_columns.empty:
        print(f"Columns with more than 50% missing values: {list(high_missing_columns)}")
        columns_to_drop.extend(high_missing_columns)

    # 2. Drop 'region' if split into 'Country' and 'Country_region'
    if 'region' in main.columns and {'Country', 'Country_region'}.issubset(main.columns):
        print("Dropping 'region' column since it has been split into 'Country' and 'Country_region'.")
        columns_to_drop.append('region')

    # 3. Identify and drop other irrelevant columns (e.g., indexing artifacts)
    irrelevant_columns = [col for col in main.columns if col.startswith('Unnamed')]
    if irrelevant_columns:
        print(f"Dropping irrelevant columns: {irrelevant_columns}")
        columns_to_drop.extend(irrelevant_columns)

    # Drop the identified columns
    if columns_to_drop:
        main.drop(columns=columns_to_drop, inplace=True, errors='ignore')
        print(f"Removed columns: {columns_to_drop}")
    else:
        print("No irrelevant columns identified for removal.")

    # Save the cleaned DataFrame
    main.to_csv("project_final_cleaned.csv", index=False)
    print("Cleaned DataFrame with dropped irrelevant columns saved to 'project_final_cleaned.csv'")
else:
    print("Error: 'main' DataFrame is not defined.")

Dropping irrelevant columns: ['Unnamed: 0']
Removed columns: ['Unnamed: 0']
Cleaned DataFrame with dropped irrelevant columns saved to 'project_final_cleaned.csv'
